In [1]:
import pandas as pd
import os
import json

In [31]:
# convert all of the cn a folder to one csv with duplicates dropped
def hist_csvs_to_csv(csv_folder, csv_file):
    # Read all CSVs from folder
    csvs = [f"{csv_folder}/{i}" for i in os.listdir(csv_folder)]
    dfs = [pd.read_csv(csv) for csv in csvs]

    # Grab required columns first
    cols = ["CARDOFFICE_CARD_NUMBER", "CARD_ID_STATUS", "RIDE_DATE", "ROUTE", "EMPLOYEE_OR_STUDENT", "CAMPUS_ID"]
    dfs = [df[cols] for df in dfs]

    # Concatenate all dataframes and drop duplicates early
    df = pd.concat(dfs).drop_duplicates()

    # Filter rows where CARD_ID_STATUS is not UNKNOWN
    df = df[df["CARD_ID_STATUS"] != "UNKNOWN"]

    # Add derived columns
    df["RIDE_DATE"] = pd.to_datetime(df["RIDE_DATE"], format="mixed")
    df["MONTH_YEAR"] = df["RIDE_DATE"].dt.strftime('%Y-%m')
    df["HOUR"] = df["RIDE_DATE"].dt.hour

    # Save to output CSV
    df.to_csv(csv_file, index=False)
    print(f"Final dataframe has {len(df)} rows.")

if __name__ == "__main__":
    hist_csvs_to_csv("./data/historical", "../public/data/historical_card_data.csv")

Final dataframe has 1395156 rows.


In [28]:
# give me the min and max date of the data
def min_max_date(df):
    min_date = df["RIDE_DATE"].min()
    max_date = df["RIDE_DATE"].max()
    return min_date, max_date

# import the csv
df = pd.read_csv("../public/data/historical_card_data.csv")
min_date, max_date = min_max_date(df)
print(min_date, max_date)
print(df.shape)

2017-01-01 06:54:00+00:00 2024-11-29 22:50:47+00:00
(1395156, 8)


In [17]:
# print out length of data
print(len(df))

1395156


In [22]:
# print size of the data
# import historical data from csv
df2 = pd.read_csv("./data/historical_data.csv")
df3 = pd.read_csv("./data/historical/historical_card_data.csv")

(1999072, 8)


In [25]:
print(df3.drop_duplicates().shape)

(1999072, 8)


In [7]:
# quickticket processing
# read all the csvs in the quickticket folder and convert them to one csv
# convert all of the csvs in a folder to one csv with duplicates dropped
def qt_csvs_to_csv(csv_folder, csv_file):
    csvs = [f"{csv_folder}/{i}" for i in os.listdir(csv_folder)]
    dfs = [pd.read_csv(csv) for csv in csvs]
    # grab certain columns and drop duplicates
    cols = ["Card Office Card Number","Campus/VUNetID","Ride Date","Route"]
    # add a column for MONTH_YEAR
    for i in range(len(dfs)):
        dfs[i] = dfs[i][cols]
        # rename these columns
        dfs[i].columns = ["CARDOFFICE_CARD_NUMBER", "CAMPUS_ID", "RIDE_DATE", "ROUTE"]
        dfs[i]["RIDE_DATE"] = pd.to_datetime(dfs[i]["RIDE_DATE"])
        dfs[i]["MONTH_YEAR"] = dfs[i]["RIDE_DATE"].dt.strftime('%Y-%m')
        # list all columns
        print(dfs[i].columns)
    df = pd.concat(dfs).drop_duplicates()
    # convert any ROUTE value == "Vandy Vans" to "Vandy Vans"
    df.loc[df["ROUTE"] == "Riverfront Station", "ROUTE"] = 100
    df.loc[df["ROUTE"] == "Donelson Station", "ROUTE"] = 101
    df.loc[df["ROUTE"] == "Hermitage Station", "ROUTE"] = 102
    df.loc[df["ROUTE"] == "Mt. Juliet Station", "ROUTE"] = 103
    df.loc[df["ROUTE"] == "Martha Station", "ROUTE"] = 104
    df.loc[df["ROUTE"] == "Hamilton Springs Station", "ROUTE"] = 105
    df.loc[df["ROUTE"] == "Lebanon Station", "ROUTE"] = 106
    
    df['CARDOFFICE_CARD_NUMBER'] = pd.to_numeric(df['CARDOFFICE_CARD_NUMBER'], errors='coerce')

    df['CARDOFFICE_CARD_NUMBER'] = df['CARDOFFICE_CARD_NUMBER'].fillna(-1).astype("int64")

    # Check the data again to ensure the conversion
    print(df.dtypes)
    
    # drop any rows with missing values
    df = df.dropna()
    print(df.columns)
    df.to_csv(csv_file, index=False)

In [8]:
if __name__ == "__main__":
    qt_csvs_to_csv("./data/quickticket", "./data/quickticket.csv")

Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')
Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')
Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')
Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')
Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')
CARDOFFICE_CARD_NUMBER             int64
CAMPUS_ID                         object
RIDE_DATE                 datetime64[ns]
ROUTE                             object
MONTH_YEAR                        object
dtype: object
Index(['CARDOFFICE_CARD_NUMBER', 'CAMPUS_ID', 'RIDE_DATE', 'ROUTE',
       'MONTH_YEAR'],
      dtype='object')


In [13]:
# import the csv
qt_df = pd.read_csv("../public/data/quickticket_data.csv")
# convert to datetime
qt_df["RIDE_DATE"] = pd.to_datetime(qt_df["RIDE_DATE"])
print(qt_df.head())
min_date = qt_df["RIDE_DATE"].min()
max_date = qt_df["RIDE_DATE"].max()
print(min_date, max_date)

  CAMPUS_ID  RIDE_DATE  ROUTE MONTH_YEAR
0    dengz2 2024-07-01      7    2024-07
1   dingay1 2024-07-01     17    2024-07
2   madrige 2024-07-01     50    2024-07
3    dengs2 2024-07-02      3    2024-07
4    dengz2 2024-07-02      7    2024-07
2024-07-01 00:00:00 2024-11-30 00:00:00


In [12]:
import csv

input_path = './data/quickticket.csv'
output_path =  '../public/data/quickticket_data.csv'

with open(input_path, 'r', newline='') as infile, open(output_path, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        # Write out rows, skipping the first column
        writer.writerow(row[1:])


In [19]:
# print out length of data
print(len(qt_df))

33679
